# Case Study Model

This notebook shows the case study results supplement to the paper "*Distribution Electricity Pricing under Uncertainty*" by Robert Mieth and Yury Dvorkin

## General Remarks

- Framework
    - Requires Julia 1.1.
    - All necessary packages with their respective versions are indicated in the Julia project/manifest files 
    - The necessary packages are automatically installed *within the environment of this project*, i.e. executing this notebook will have no effect on globally installed packages and versions of your Julia distribution


- Reverse Power Flow / Cost at substation
    - If the negative generation limit at the substation is not enforced, then substation has the option of pushing energy back towards the transmission system (reverse power flow)
    - We make the follwoing assumptions: 
        - reverse power flow is allowed at the substation 
        - for scheduled negative active power the substation is remunerated according to the market price (i.e. linear cost of energy provision at the substation) -> the substation has a linear cost function 
        - While the cost function will not have a quadratic termn, THERE WILL BE a penalty on $\alpha_0$ as it indicates how much the substation is willing to deviate from the schedule
        
- Implementational Notes
    - cost function of all generators in this implementation follows the form c_2\*g^2 + c_1\*g + c_0 
    - Quadratic objective has been reformulated in a linear objective with second-order cone constraint. The quantiative values are identical, see `model_definition.jl` for details

## Notebook Set-Up and Packages

In [1]:
# increase the width of the notebook to improve result table visibility
display("text/html", "<style>.container { width:80% !important; }</style>")

In [3]:
# Set up environment from .toml-files
import Pkg
Pkg.activate(".")
Pkg.instantiate() # This line can be deactivated after first run to speed up the process

# Load necessary packages
using DataFrames, CSV, LinearAlgebra, Dates # Basic Julia utility
using JLD, HDF5 # For saving the parameters 
using JuMP # Modeling Framework
using Mosek, MosekTools # Solver and Solver-Environment
using Logging # For some warnings in the functions

# Load functions and model
include("src/tools.jl") # Some additional functions
include("src/input.jl") # Type definitions and read-in functions
include("src/model_definition.jl") # Model definiton
include("src/output.jl") # Postprocessing of solved model
;

  Updating registry at `C:\Users\Robert\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


# 15Bus Case

- 15 bus test feeder from "Analysis of distribution locational marginal prices" (IEEE TSG 2018) by Anthony Papavasilou
        
- Assumptions for all cases except mentioned differently
    - 
    - Two DERs one each at node 6 and 11 with `c1 = 10`, `c2 = 5`, `p_max = 1`, `q_max = 0.5`
    - No reverse power flow limit on substation
    - Substation pays `c1 = 50` for energy provision from the transmission network (no quadratic term) and a penalty of `200` on alpha to discourage high alpha
    - All nodes have 20% standard deviation of their forecasted load
    - Lines 2 and 3 have double the thermal capacity

## Experiment definition

- Tag: `experiment_15b`

- Runs:
    - `no_uncert`: deterministic case (det)
    - `uncert`: case with uncertainty (VOLT_CC)
    - `gen_uncert`: chance constraints only in generation (GEN_CC)



In [10]:
experiment_15b = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/feeder15/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = abs.(loads) .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex15b_general_settings
ex15b_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945, # 95% quantile
    "z_v" => 2.576, # 99% quantile
    "z_f" => 2.576, # 99% quantile
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "toggle_thermal_cc" => false,
    "thermal_const_method" => 2, #1=soc, 2=inner approx
    "vfac" => 0, # directly sets symetric voltage limit to min:(1-vfac) max:(1+vfac), use import values if vfac=0
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
)

# ex15b_no_uncert:
ex15b_no_uncert_settings = copy(ex15b_general_settings)
ex15b_no_uncert_settings["toggle_volt_cc"] = false
ex15b_no_uncert_settings["toggle_gen_cc"] = false
ex15b_no_uncert_settings["toggle_thermal_cc"] = false

experiment_15b["no_uncert"] = Dict(
    "verbose" => "no_uncert",
    "feeder" => feeder,
    "settings" => ex15b_no_uncert_settings,
)

# ex15b_uncert:
ex15b_uncert_settings = copy(ex15b_general_settings)
ex15b_uncert_settings["toggle_volt_cc"] = true
ex15b_uncert_settings["toggle_gen_cc"] = true
ex15b_uncert_settings["toggle_thermal_cc"] = false
experiment_15b["uncert"] = Dict(
    "verbose" => "uncert",
    "feeder" => feeder,
    "settings" => ex15b_uncert_settings,
)

# ex15b_gen_uncert:
ex15b_gen_uncert_settings = copy(ex15b_general_settings)
ex15b_gen_uncert_settings["toggle_volt_cc"] = false
ex15b_gen_uncert_settings["toggle_gen_cc"] = true
ex15b_gen_uncert_settings["toggle_thermal_cc"] = false
experiment_15b["gen_uncert"] = Dict(
    "verbose" => "gen_uncert",
    "feeder" => feeder,
    "settings" => ex15b_gen_uncert_settings,
)

# Run 
experiment_15b = run_experiment(experiment_15b)
;

>>>>> Reading feeder data from data/feeder_data/feeder15/
Running uncert
>>>> Building Model
>>>> Running Model
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(86) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(92) of matrix 'A'.
>>>> Model finished with status OPTIMAL in 0.014609778 seconds
>>>> Post-Processing

Running gen_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.007189333 seconds
>>>> Post-Processing

Running no_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.004408889 seconds
>>>> Post-Processing



## Results

In [5]:
# Deterministic Case
display(experiment_15b["no_uncert"]["results"])

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,nu,delta_plus,delta_minus,mu_plus,mu_minus,eta_plus,eta_minus,eta_aP,eta_aQ,voltvar,lambda_anc,rx_pi_i,rx_pi_a,rx_etaQ,r_sum_mu_d,nu_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,61.5275,1,1.1833,0.3237,1.0,0,0,0,-50.0,-1.3543e-9,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.976936,0.5376,0.1855,0,-50.0,-1.98615e-7,0.0,0,0,0,0,-9.30013e-8,-1.68856e-7,0.0,0,0.0,0.0,0,-50.0,-1.65512e-9,-1.12858e-11,0.0,-1.66641e-9,0
3,0.0,3,0,0,0.999807,-0.256,1.95709e-11,0,-30.1628,-5.3144,0.0,0,0,0,0,-1.16915e-7,-1.30033e-7,-19.8372,0,19.8372,-5.3144,0,-50.0,-3.71839,-1.38967e-7,-3.71839,-1.6054e-7,0
4,0.0,4,0,0,1.03464,-0.256,1.95709e-11,0,-10.3255,-2.54011e-8,0.0,0,0,0,0,-1.78992e-7,-9.52812e-8,-19.8372,0,19.8372,5.3144,0,-30.1628,-1.77731e-8,-3.71847,3.71847,-2.55259e-7,0
5,0.0,5,0,0,1.0357,-0.0460628,-0.00820303,0,-10.3255,-1.03056e-8,0.0,0,0,0,0,-1.8256e-7,-9.42453e-8,-2.41589e-7,0,-1.07809e-7,-5.21328e-8,0,-10.3255,-7.21015e-9,-1.77715e-8,-3.64739e-8,-1.14922e-8,0
6,0.0,6,0,0,1.03708,-0.0633628,-0.012503,0,-10.3255,-3.63395e-10,0.0,0,0,0,0,-1.87093e-7,-9.30231e-8,0.0,0,0.0,0.0,0,-10.3255,-2.53363e-10,-7.18518e-9,0.0,-7.43855e-9,0
7,0.0,7,0.114363,0.025303,1.04267,-0.0924628,-0.019803,0,-10.3255,-5.7619e-8,0.0,0,0,0.0,-1.97491e-7,-2.07035e-7,-8.85744e-8,-7.55592e-7,0,5.94469e-7,4.09317e-8,0,-10.3255,-4.03082e-8,-2.54219e-10,2.86344e-8,-1.14196e-8,0
8,0.0,8,0,0,1.05334,-0.1969,0.0019,0,-10.3255,-1.14614e-7,0.0,0,0,0,0,-2.46517e-7,0.0,0.0,0,0.0,0.0,0,-10.3255,-8.02453e-8,-5.44596e-8,0.0,-2.57857e-8,0
9,0.0,9,0,0,1.04366,-0.230037,-0.000196966,0,-10.3255,-7.77846e-8,0.0,0,0,0,0,0.0,-8.92158e-8,-1.67111e-6,0,1.48677e-6,-1.01958e-8,0,-10.3255,-5.43958e-8,-1.77633e-8,-7.13006e-9,-4.37625e-8,0


In [6]:
# GEN-CC 
display(experiment_15b["gen_uncert"]["results"])

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,nu,delta_plus,delta_minus,mu_plus,mu_minus,eta_plus,eta_minus,eta_aP,eta_aQ,voltvar,lambda_anc,rx_pi_i,rx_pi_a,rx_etaQ,r_sum_mu_d,nu_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,70.4343,1,1.1833,0.3237,1.0,0,0,0.428602,-50.0,-5.40272e-9,-19.8899,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.976936,0.5376,0.1855,0,-50.0,-7.12378e-7,0.0,0,0,0,0,-3.32182e-7,-6.04708e-7,0.0,0,0.0,0.0,0,-50.0,-5.93648e-9,-4.50227e-11,0.0,-5.98151e-9,0
3,0.0,3,0,0,0.999807,-0.256,2.65741e-11,0,-6.16131,-11.7444,0.0,0,0,0,0,-4.19262e-7,-4.66164e-7,-43.8387,0,43.8387,-11.7444,0,-50.0,-8.21735,-4.98439e-7,-8.21735,-5.76295e-7,0
4,0.0,4,0,0,1.03464,-0.256,2.65741e-11,0,-37.6774,-3.82881e-7,0.0,0,0,0,0,-6.4309e-7,-3.41806e-7,-43.8387,0,43.8387,11.7444,0,-6.16131,-2.67901e-7,-8.21751,8.21751,-9.16258e-7,0
5,0.0,5,0,0,1.03576,-0.0461079,-0.0102331,0,-37.6774,-2.86456e-7,0.0,0,0,0,0,-6.56648e-7,-3.37952e-7,-1.63259e-6,0,-1.15351e-6,3.68639e-8,0,-37.6774,-2.00414e-7,-2.67877e-7,2.57913e-8,-4.16709e-8,0
6,0.0,6,0,0,1.03718,-0.0634079,-0.0145331,0,-37.6774,-2.47693e-7,0.0,0,0,0,0,-6.73747e-7,-3.33442e-7,0.0,0,0.0,0.0,0,-37.6774,-1.72695e-7,-1.9972e-7,0.0,-2.70258e-8,0
7,0.0,7,0.114408,0.0273331,1.04291,-0.0925079,-0.0218331,0.285719,-37.6774,-3.0918e-8,0.0,0,0,-1.44048e-7,-49.2583,-7.49022e-7,-3.17162e-7,-2.69938e-6,0,2.11952e-6,1.57265e-7,0,-37.6774,-2.16292e-8,-1.73278e-7,1.10017e-7,-4.16314e-8,0
8,0.0,8,0,0,1.05323,-0.1969,0.0019,0,-37.6774,-5.53377e-8,0.0,0,0,0,0,-8.82708e-7,0.0,0.0,0,0.0,0.0,0,-37.6774,-3.87438e-8,-1.31075e-7,0.0,-9.23312e-8,0
9,0.0,9,0,0,1.04354,-0.229992,0.00183313,0,-37.6774,-1.87214e-7,0.0,0,0,0,0,0.0,-3.20248e-7,-4.74601e-6,0,4.08618e-6,-2.75924e-8,0,-37.6774,-1.30921e-7,-2.67754e-7,-1.92957e-8,-1.56128e-7,0


In [7]:
# VOLT-CC 
display(experiment_15b["uncert"]["results"])

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,nu,delta_plus,delta_minus,mu_plus,mu_minus,eta_plus,eta_minus,eta_aP,eta_aQ,voltvar,lambda_anc,rx_pi_i,rx_pi_a,rx_etaQ,r_sum_mu_d,nu_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,71.7329,1,1.1981,0.378944,1.0,0,0,0.465563,-50.0,-3.32827e-9,-19.9181,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.970112,0.5524,0.240744,0,-49.963,-4.43619,0.0,0.000534437,-0.018304,0,0,-1.98278e-7,-7.43554e-7,0.0,0,0.0,0.0,0.000255774,-50.0,-0.0369682,-2.77356e-11,0.0,-0.0369682,-0.0183041
3,0.0,3,0,0,0.984769,-0.2412,0.0552439,0,-19.9972,-8.76626,0.0,0.0477252,-1.63454,0,0,-2.71298e-7,-6.51034e-7,-66.6959,0,66.6959,-17.8678,0.000414416,-49.963,-6.1336,-3.10393,-12.5018,-3.2643,-1.63455
4,0.0,4,0,0,1.00731,-0.2412,0.0552439,0,-25.1136,-1.45394,0.0,0.121691,-4.16781,0,0,-1.04925e-6,-8.30509e-7,-2.44555e-6,0,1.29254e-6,3.92048e-7,0.000990722,-19.9972,-1.01732,-6.13372,2.74315e-7,-5.1164,-4.16784
5,0.0,5,0,0,1.00937,-0.0372927,-0.0498092,0,-25.3425,-1.12672,0.0,0.126728,-4.28435,0,0,-1.32613e-6,-8.19686e-7,-7.25137e-7,0,-5.14136e-7,7.94197e-8,0.00103784,-25.1136,-0.788289,-1.01723,5.55647e-8,-0.228939,-4.28437
6,0.0,6,0,0,1.01166,-0.0545927,-0.0541092,0,-25.5523,-0.825859,0.0,0.131343,-4.39113,0,0,-1.78832e-6,-7.90239e-7,0.0,0,0.0,0.0,0.0010757,-25.3425,-0.575798,-0.785559,0.0,-0.209761,-4.39115
7,0.0,7,0.105593,0.0669092,1.01979,-0.0836927,-0.0614092,0.263704,-26.13,-3.13706e-8,0.0,0.144053,-4.68522,-5.15303e-8,-37.69,-5.99317,-6.37813e-7,-1.07436e-6,0,8.23974e-7,1.66261e-7,0.00108905,-25.5523,-2.19458e-8,-0.577742,1.1631e-7,-0.577742,-4.68523
8,0.0,8,0,0,1.02077,-0.1969,0.0019,0,-27.4369,-1.86615,0.0,0.13271,-5.30265,0,0,-12.4909,-5.87498e-7,0.0,0,0.0,0.0,0.00106365,-26.1304,-1.30655,-1.59777e-7,0.0,-1.30655,-5.30269
9,0.0,9,0,0,1.01077,-0.224007,0.0966531,0,-26.1304,-2.2821e-7,0.0,0.13271,-4.66446,0,0,-2.03412e-6,-8.88834e-7,-7.13656e-6,0,6.77284e-6,-2.30366e-6,0.0011173,-25.1136,-1.5959e-7,-1.01676,-1.61098e-6,-1.01676,-4.66449


In [11]:
save_experiment("experiment_15b", experiment_15b)

Saving uncert
Saving gen_uncert
Saving no_uncert
>>>> Saved with timestamp 190422_1211
